In [1]:
# The lovely imports
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("display.max_colwidth", None)

### PART OF SPEECH TAGGING (POS)

In [3]:
with tf.device('/GPU:0'):
    # Load the spacy model
    nlp = spacy.load('en_core_web_md')

    # Reading the csv file with the data
    df = pd.read_csv('../Task_5/datasets/dataset_v2.csv')

    # New column for the POS tags
    df['POS_Tags'] = df['text'].apply(lambda x: [token.pos_ for token in nlp(x)])

In [ ]:
# Checking the first 5 rows
df['POS_Tags'].head()

0                                                                           [PRON, AUX, ADV, VERB, PRON, ADV, CCONJ, PRON, ADV, AUX, VERB, ADJ, ADP, PRON, ADP, ADV, VERB, ADP, ADP, PRON, ADJ, NOUN, NOUN, ADP, ADJ, CCONJ, ADJ, NOUN]
1                                                                                                                                 [PRON, VERB, SCONJ, PRON, AUX, ADV, ADV, ADJ, ADP, ADJ, NOUN, NOUN, CCONJ, VERB, PART, AUX, ADV, ADJ]
2    [PRON, VERB, PRON, ADJ, ADP, NOUN, SCONJ, PRON, VERB, SCONJ, PRON, VERB, ADV, DET, NOUN, ADP, VERB, NUM, DET, AUX, ADV, ADP, DET, ADJ, CCONJ, VERB, ADP, DET, ADJ, CCONJ, PRON, AUX, VERB, SCONJ, PRON, AUX, PART, ADV, DET, NOUN]
3                                                                                        [PRON, AUX, NUM, ADP, DET, NOUN, PRON, VERB, ADP, VERB, ADP, DET, NOUN, AUX, ADV, ADJ, SCONJ, PRON, AUX, AUX, ADV, ADP, DET, NOUN, CCONJ, ADJ]
4                                                                       

### TERM FREQUENCY-INVERSE DOCUMENT FREQUENCY (TF-IDF)

In [ ]:
with tf.device('/GPU:0'):
    # Create the vectorizer
    vectorizer = TfidfVectorizer()
    
    # Fit the vectorizer and transform the data
    X = vectorizer.fit_transform(df['text'])
    
    # Get feature names (words)
    feature_names = vectorizer.get_feature_names_out()
    
    # Create a word-to-index dictionary
    word_to_index = {word: idx for idx, word in enumerate(feature_names)}
    
    # Convert TF-IDF matrix to a list of (index, value) tuples per row
    def sparse_to_tuple(row, word_to_index, feature_names):

        return [(word_to_index[feature_names[col]], value) for col, value in zip(row.indices, row.data)]
    
    df['TF_IDF'] = [sparse_to_tuple(X[i], word_to_index, feature_names) for i in range(X.shape[0])]


In [6]:
# Print result
df

text  \
0                                                                      ive probably mentioned this before but i really do feel proud of myself for actually keeping up with my new years resolution of monthly and weekly goals   
1                                                                                                                           i beleive that i am much more sensitive to other peoples feelings and tend to be more compassionate   
2       i find myself frustrated with christians because i feel that there is constantly a talk about loving one another being there for each other and praying for each other and i have seen that this is not always the case   
3                                                                                                           i am one of those people who feels like going to the gym is only worthwhile if you can be there for an hour or more   
4                                                                                                                                                      i feel especially pleased about this as this has been a long time coming   
...                                                                                                                                                                                                                         ...   
117255                                                                                                                                                           Who was it saying that [NAME] destroys hooker and broner again   
117256                                                                                                                  Just hearing that thing's carapace sliding around and it's feet hitting the ground would make me vomit.   
117257                                                                                             Yeeeaaahhh.... "protect"... I'm pretty sure this guy has a rape dungeon filled with schoolgirl body-pillows in his basement.   
117258                                                                                                                                                               [NAME] can’t space and [NAME] is getting worse defensively   
117259                                                                                                                                                              That sounds awful. Glad its from 4Chan, so I know its fake.   

       dominant_emotion  \
0             happiness   
1             happiness   
2             happiness   
3             happiness   
4             happiness   
...                 ...   
117255          disgust   
117256          disgust   
117257          disgust   
117258          disgust   
117259          disgust   

                                                                                                                                                                                                                                  POS_Tags  \
0                                                                              [PRON, AUX, ADV, VERB, PRON, ADV, CCONJ, PRON, ADV, AUX, VERB, ADJ, ADP, PRON, ADP, ADV, VERB, ADP, ADP, PRON, ADJ, NOUN, NOUN, ADP, ADJ, CCONJ, ADJ, NOUN]   
1                                                                                                                                    [PRON, VERB, SCONJ, PRON, AUX, ADV, ADV, ADJ, ADP, ADJ, NOUN, NOUN, CCONJ, VERB, PART, AUX, ADV, ADJ]   
2       [PRON, VERB, PRON, ADJ, ADP, NOUN, SCONJ, PRON, VERB, SCONJ, PRON, VERB, ADV, DET, NOUN, ADP, VERB, NUM, DET, AUX, ADV, ADP, DET, ADJ, CCONJ, VERB, ADP, DET, ADJ, CCONJ, PRON, AUX, VERB, SCONJ, PRON, AUX, PART, ADV, DET, NOUN]   
3                                                                                           [PRON, AUX, NUM, ADP, DET, NOUN, PRON, VERB, ADP, VERB, ADP, DET, NOUN, AUX, ADV, ADJ, SCONJ, PRON, AUX, AUX, ADV, ADP, DET, 

In [6]:
df['TF_IDF'].head()

0                                                                                                                           {'ive': 0.15881804994572354, 'probably': 0.20387707513971629, 'mentioned': 0.26601995122707073, 'this': 0.10798705575539878, 'before': 0.18346493639354625, 'but': 0.10573294021220193, 'really': 0.13788043688484908, 'do': 0.1330776090833255, 'feel': 0.0570753946507701, 'proud': 0.23833130608288738, 'of': 0.17057266825527476, 'myself': 0.14447506983534475, 'for': 0.1075388701552226, 'actually': 0.18290691721370506, 'keeping': 0.2633177396995653, 'up': 0.13626352711638753, 'with': 0.11157960582570181, 'my': 0.0923971840650495, 'new': 0.18970655027846364, 'years': 0.194565065484257, 'resolution': 0.33370179646921283, 'monthly': 0.35611442898158413, 'and': 0.06724231572902031, 'weekly': 0.32907348792590335, 'goals': 0.2836476128225077}
1                                                                                                                                      

### SENTIMENT ANALYSIS

In [ ]:
with tf.device('/GPU:0'):
    # Define a function to get sentiment score using TextBlob
    def get_sentiment(text):
        blob = TextBlob(text)
        return blob.sentiment.polarity

    # Apply the function to the text column
    df['Sentiment_Score'] = df['text'].apply(get_sentiment)

    # Check the first 5 rows of the sentiment score
    df['Sentiment_Score'].head()

### PRETRAINED WORD EMBEDDINGS

In [ ]:
# Importing the necessary libraries for embedding
import numpy as np

with tf.device('/GPU:0'):
    # Generating numerical vector representations
    df['Pretrained_Embeddings'] = df['text'].apply(
        lambda x: nlp(x).vector if isinstance(x, str) and x.strip() else np.zeros(nlp.vocab.vectors_length)
    )
    # Check the first 5 rows of the pretrained embeddings
    df['Pretrained_Embeddings'].head()

### CUSTOM WORD EMBEDDINGS

In [10]:
with tf.device('/GPU:0'):
    sentences = df['text'].tolist()

    # Ensure all entries are strings
    sentences = [str(s) if pd.notna(s) else "" for s in sentences]

    # Tokenize words
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    word_index = tokenizer.word_index
    vocab_size = len(word_index) + 1  # +1 for padding

    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    max_length = max(len(seq) for seq in sequences) if sequences else 1
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

    # Define word embedding model
    embedding_dim = 50  # Size of each word vector
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
        Flatten(),
        Dense(embedding_dim, activation='relu')  # Output layer
    ])

    # Compile and train the model
    model.compile(optimizer='adam', loss='mse')
    model.fit(padded_sequences, np.zeros((len(sentences), embedding_dim)), epochs=50, verbose=0)

    # Extract the trained embedding layer
    word_embeddings = model.layers[0].get_weights()[0]

### Additional feature (Stopword Removal) - Removes common words that don’t contribute to meaning.

In [11]:
with tf.device('/GPU:0'):
    # Function to compute average word embedding for a sentence
    def get_sentence_embedding(sentence):
        if pd.isna(sentence):
            return np.zeros(embedding_dim)
        
        words = str(sentence).split()
        word_vectors = [word_embeddings[word_index[word]] for word in words if word in word_index]
        
        if word_vectors:
            return np.mean(word_vectors, axis=0)  # Average of word embeddings
        else:
            return np.zeros(embedding_dim)  # Return zero vector if no words are found

    # Apply function to each sentence
    df["Custom_Embeddings"] = df["text"].apply(get_sentence_embedding)

    # Stopword Removal
    def remove_stopwords(text):
        if pd.isna(text):
            return ""
        doc = nlp(str(text))
        return " ".join([token.text for token in doc if not token.is_stop])

    df["Cleaned_Text"] = df["text"].apply(remove_stopwords)

In [12]:
df

,text,dominant_emotion,POS_Tags,TF_IDF,Sentiment_Score,Pretrained_Embeddings,Custom_Embeddings,Cleaned_Text
0,ive probably mentioned this before but i really do feel proud of myself for actually keeping up with my new years resolution of monthly and weekly goals,happiness,"[PRON, AUX, ADV, VERB, PRON, ADV, CCONJ, PRON, ADV, AUX, VERB, ADJ, ADP, PRON, ADP, ADV, VERB, ADP, ADP, PRON, ADJ, NOUN, NOUN, ADP, ADJ, CCONJ, ADJ, NOUN]","{'ive': 0.15881804994572354, 'probably': 0.20387707513971629, 'mentioned': 0.26601995122707073, 'this': 0.10798705575539878, 'before': 0.18346493639354625, 'but': 0.10573294021220193, 'really': 0.13788043688484908, 'do': 0.1330776090833255, 'feel': 0.0570753946507701, 'proud': 0.23833130608288738, 'of': 0.17057266825527476, 'myself': 0.14447506983534475, 'for': 0.1075388701552226, 'actually': 0.18290691721370506, 'keeping': 0.2633177396995653, 'up': 0.13626352711638753, 'with': 0.11157960582570181, 'my': 0.0923971840650495, 'new': 0.18970655027846364, 'years': 0.194565065484257, 'resolution': 0.33370179646921283, 'monthly': 0.35611442898158413, 'and': 0.06724231572902031, 'weekly': 0.32907348792590335, 'goals': 0.2836476128225077}",0.284091,"[-0.75738746, 0.2743086, -0.20282078, -0.16432592, -0.17436278, -0.09778982, 0.053379975, -0.17496917, 0.10653638, 2.1026115, -0.26742008, 0.07979832, -0.046994973, 0.0302029, -0.15144645, -0.020129427, -0.034120645, 0.85209477, -0.20020247, 0.04653565, -0.0957774, 0.01676632, 0.008201099, -0.18804303, 0.05499732, -0.079989254, -0.14925566, -0.104818396, 0.13417105, -0.11536827, -0.09862103, 0.04417054, -0.13780302, -0.025186356, 0.046723492, 0.2220984, 0.010555425, -0.046409465, 0.01792274, 0.07372361, 0.0019162871, 0.03005943, 0.031134412, 0.009844351, -0.0031047128, -0.045497216, 0.0006266066, 0.13341756, -0.033141464, 0.05488291, -0.12176339, 0.097648494, 0.09347433, -0.036310043, 0.046934273, 0.0011488636, -0.023298997, -0.16003692, 0.0064012804, -0.08205436, -0.14347515, -0.13707088, -0.12366926, 0.12699603, 0.24494828, -0.042354167, -0.010621716, 0.034852996, 0.06841698, -0.10183527, 0.08120461, 0.13026671, 0.1351739, -0.11074477, -0.022072634, 0.17193775, 0.15181674, -0.06520503, 0.0037726786, 0.30456546, -0.022543276, 0.093277946, 0.005578037, -0.077440366, -0.10490926, -0.11316712, 0.3552496, -0.2487334, 0.11552091, -0.13273081, -0.095897816, -0.049685545, -0.13305397, -0.012673532, -0.0431176, -0.12085386, -0.023925645, -0.030869136, 0.10087853, 0.10073364, ...]","[-0.0016498204, -0.00375087, 0.0007556699, 0.0090068, -0.0005878196, 0.00774577, 0.009944604, 0.009495123, 0.0051781093, 0.0014239594, 0.0055776364, -0.0023322818, -0.009361548, 0.0040879184, 0.0004417717, 0.009805979, 0.00044469454, 0.0045996057, 0.0011167153, 0.011732154, 0.002681814, 0.0044779405, -0.00031457993, 0.002397936, -0.008689696, -0.0026094378, -0.0042171488, -0.0006424266, 0.0008338726, 0.0045282864, -0.0051994747, 0.0006787064, -0.0027190468, 2.2879905e-05, 0.00095903245, 0.00036020577, -0.0013424634, -0.0034989843, 0.0005216959, -0.008172285, -0.0060227416, -0.0031910902, -0.001120724, -0.0039096996, 0.014694726, 0.004151856, -0.008712682, 0.0024986132, -0.008852518, 0.0027358294]",ve probably mentioned feel proud actually keeping new years resolution monthly weekly goals
1,i beleive that i am much more sensitive to other peoples feelings and tend to be more compassionate,happiness,"[PRON, VERB, SCONJ, PRON, AUX, ADV, ADV, ADJ, ADP, ADJ, NOUN, NOUN, CCONJ, VERB, PART, AUX, ADV, ADJ]","{'and': 0.08350415233657166, 'beleive': 0.45087440860247063, 'that': 0.10464189182075356, 'am': 0.1441680346102422, 'much': 0.19373048684358654, 'more': 0.34276862764590604, 'sensitive': 0.3498920897516954, 'to': 0.17265683293504636, 'other': 0.2109040823528817, 'peoples': 0.34047083529653777, 'feelings': 0.222106514192201, 'tend': 0.314080869966579, 'be': 0.14453113994840147, 'compassionate': 0.34853983680613304}",0.243750,"[-0.7262283, 0.112184614, -0.37276185, -0.06319817, -0.08333776, 0.09231962

### Saving it into a csv

In [ ]:
with tf.device('/GPU:0'):
    # Save the DataFrame to a CSV file
    df.to_csv("../Task_5/datasets/dataset_v2.csv", index=False)